# Adaptive Rounding (AdaRound)
This notebook shows a working code example of how to use AIMET to perform Adaptive Rounding (AdaRound).

AIMET quantization features typically use the "nearest rounding" technique for achieving quantization.
When using the "nearest rounding" technique, the weight value is quantized to the nearest integer value.

AdaRound optimizes a loss function using unlabeled training data to decide whether to quantize a specific weight to the closer integer value or the farther one.
Using AdaRound quantization, a model is able to achieve an accuracy closer to the FP32 model, while using low bit-width integer quantization.

#### Overall flow
This notebook covers the following:
1. Instantiate the example evaluation and training pipeline
2. Load the FP32 model and evaluate the model to find the baseline FP32 accuracy
3. Create a quantization simulation model (with fake quantization ops inserted) and evaluate this simuation model to get a quantized accuracy score
4. Apply AdaRound and evaluate the simulation model to get a post-finetuned quantized accuracy score

#### What this notebook is not
* This notebook is not designed to show state-of-the-art results
* For example, it uses a relatively quantization-friendly model like Resnet18
* Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly

---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification.
If you already have a version of the dataset readily available, use that.
Otherwise, download the dataset from an appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The dataloader provided in this example notebook relies on the ImageNet tfrecords dataset having the following characteristics:
- Subfolders 'train' for the training samples and 'val' for the validation samples.
Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample.

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset.
E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class.
But for the purpose of running this notebook, you could reduce the dataset to 2 samples per class.
This exercise is left up to the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [ ]:
DATASET_DIR = '/path/to/dataset/'         # Please replace this with a real directory

---
## 1. Example evaluation and training pipeline

The following is an example training and validation loop for this image classification task.

- **Does AIMET have any limitations on how the training, validation pipeline is written?** Not really. We will see later that AIMET will modify the user's model to create a QuantizationSim model which is still a PyTorch model. This QuantizationSim model can be used in place of the original model when doing inference or training.
- **Does AIMET put any limitation on the interface of the evaluate() or train() methods?** Not really. You should be able to use your existing evaluate and train routines as-is.


In [ ]:
import os
import torch
from Examples.common import image_net_config
from Examples.torch.utils.image_net_evaluator import ImageNetEvaluator
from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

class ImageNetDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(DATASET_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         is_training=False,
                                         num_workers=image_net_config.evaluation['num_workers']).data_loader
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = ImageNetEvaluator(DATASET_DIR, image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      num_workers=image_net_config.evaluation['num_workers'])

        return evaluator.evaluate(model, iterations=None, use_cuda=use_cuda)

---
## 2. Load the model and evaluate to get a baseline FP32 accuracy score

For this example notebook, we are going to load a pretrained resnet18 model from torchvision.
Similarly, you can load any pretrained PyTorch model instead.

In [ ]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)

AIMET quantization simulation requires the user's model definition to follow certain guidelines.
For example, functionals defined in forward pass should be changed to equivalent torch.nn.Module.
AIMET user guide lists all these guidelines.
The following **ModelPreparer** API uses new graph transformation feature available in PyTorch 1.9+ version and automates model definition changes required to comply with the above guidelines. 

In [ ]:
from aimet_torch.model_preparer import prepare_model

model = prepare_model(model)

---
We should decide whether to place the model on a CPU or CUDA device.
This example code will use CUDA if available in your current execution environment.
You can change this logic and force a device placement if needed.

In [ ]:
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.to(torch.device('cuda'))

---
Let's determine the FP32 (floating point 32-bit) accuracy of this model using the evaluate() routine

In [ ]:
accuracy = ImageNetDataPipeline.evaluate(model, use_cuda)
print(accuracy)

---
## 3. Create a quantization simulation model and determine quantized accuracy


## Fold Batch Normalization layers
Before we determine the simulated quantized accuracy using QuantizationSimModel, we will fold the BatchNormalization (BN) layers in the model.
These layers get folded into adjacent Convolutional layers. The BN layers that cannot be folded are left as they are.

**Why do we need to this?**

On quantized runtimes (like TFLite, SnapDragon Neural Processing SDK, etc.), it is a common practice to fold the BN layers.
Doing so results in an inferences/sec speedup since unnecessary computation is avoided.

From a floating point compute perspective, a BN-folded model is mathematically equivalent to a model with BN layers from an inference perspective, and produces the same accuracy.
However, folding the BN layers can increase the range of the tensor values for the weight parameters of the adjacent layers.

This can have a negative impact on the quantized accuracy of the model (especially when using INT8 or lower precision).
We want to simulate that on-target behavior by doing BN folding here.

The following code calls AIMET to fold the BN layers in-place on the given model:

In [ ]:
from aimet_torch.batch_norm_fold import fold_all_batch_norms

_ = fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

In [ ]:
from aimet_common.defs import QuantScheme
from aimet_torch.quantsim import QuantizationSimModel

dummy_input = torch.rand(1, 3, 224, 224)    # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)
if use_cuda:
    dummy_input = dummy_input.cuda()

sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

---
We can check the modifications AIMET has made to the model graph.
One way is to print the model, and we can see that AIMET has added quantization wrapper layers.

**Note**: use sim.model to access the modified PyTorch model.
By default, AIMET creates a copy of the original model prior to modifying it.
There is a parameter to override this behavior.

In [ ]:
print(sim.model)

---
We can also check how AIMET has configured the added fake quantization nodes, which AIMET refers to as 'quantizers'.
You can see this by printing the sim object.

In [ ]:
print(sim)

---
Even though AIMET has added 'quantizer' nodes to the model graph, the model is not ready to be used yet.
Before we can use the sim model for inference or training, we need to find appropriate scale/offset quantization parameters for each 'quantizer' node.

For activation quantization nodes, we need to pass unlabeled data samples through the model to collect range statistics which will then let AIMET calculate appropriate scale/offset quantization parameters.
This process is sometimes referred to as calibration. AIMET simply refers to it as 'computing encodings'.

We create a routine to pass unlabeled data samples through the model.
This should be fairly simple - use the existing train or validation data loader to extract some samples and pass them to the model.
We don't need to compute any loss metrics, so we can just ignore the model output. A few pointers regarding the data samples:

- In practice, we need a very small percentage of the overall data samples for computing encodings.
  For example, the training dataset for ImageNet has 1M samples. For computing encodings we only need 500 to 1000 samples.
- It may be beneficial if the samples used for computing encoding are well distributed.
  It's not necessary that all classes need to be covered since we are only looking at the range of values at every layer activation.
  However, we definitely want to avoid an extreme scenario like all 'dark' or 'light' samples are used - e.g. only using pictures captured at night might not give ideal results.

The following shows an example of a routine that passes unlabeled samples through the model for computing encodings.
This routine can be written in many different ways, this is just an example.

In [ ]:
def pass_calibration_data(sim_model, use_cuda):
    data_loader = ImageNetDataPipeline.get_val_dataloader()
    batch_size = data_loader.batch_size

    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    samples = 1000

    batch_cntr = 0
    with torch.no_grad():
        for input_data, target_data in data_loader:

            inputs_batch = input_data.to(device)
            sim_model(inputs_batch)

            batch_cntr += 1
            if (batch_cntr * batch_size) > samples:
                break

---
Now we call AIMET to use the above routine to pass data through the model and then subsequently compute the quantization encodings.
Encodings here refer to scale/offset quantization parameters.

In [ ]:
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

---
Now the QuantizationSim model is ready to be used for inference.
First we can pass this model to the same evaluation routine we used before.
The evaluation routine will now give us a simulated quantized accuracy score for INT8 quantization instead of the FP32 accuracy score we saw before.

In [ ]:
accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

---
## 4. Apply Adaround

We can now apply AdaRound to this model.

Some of the parameters for AdaRound are described below

- **dataloader:**  AdaRound needs a dataloader to use data samples for the layer-by-layer optimization to learn the rounding vectors. Either a training or validation dataloader could be passed in.
- **num_batches:** The number of batches used to evaluate the model while calculating the quantization encodings. Typically we want AdaRound to use around 2000 samples. So with a batch size of 32, this may translate to 64 batches. To speed up the execution here we are using a batch size of 1.
- **default_num_iterations:** The number of iterations to adaround each layer. Default value is set to 10000 and we strongly recommend to not reduce this number. But in this example we are using 32 to speed up the execution runtime.

In [ ]:
from aimet_torch.adaround.adaround_weight import Adaround, AdaroundParameters

data_loader = ImageNetDataPipeline.get_val_dataloader()
params = AdaroundParameters(data_loader=data_loader, num_batches=1, default_num_iterations=32)

dummy_input = torch.rand(1, 3, 224, 224)
if use_cuda:
    dummy_input = dummy_input.cuda()

os.makedirs('./output/', exist_ok=True)
ada_model = Adaround.apply_adaround(model, dummy_input, params,
                                    path="output", 
                                    filename_prefix='adaround', 
                                    default_param_bw=8,
                                    default_quant_scheme=QuantScheme.post_training_tf_enhanced)

---
Now, we can determine the simulated quantized accuracy of the model after applying Adaround.
We again create a simulation model like before and evaluate to determine simulated quantized accuracy.

**Note:** There are two important things to understand in the following cell.
  - **Parameter Biwidth Precision**: The QuantizationSimModel must be created with the same parameter bitwidth precision that was used in the apply_adaround() created.
    
  - **Freezing the parameter encodings**:
After creating the QuantizationSimModel, the set_and_freeze_param_encodings() API must be called before calling the compute_encodings() API.
While applying AdaRound, the parameter values have been rounded up or down based on these initial encodings internally created.
For Quantization Simulation accuracy, it is important to freeze these encodings.
If the parameters encodings are NOT frozen, the call to compute_encodings() will alter the value of the parameters encodings and Quantization Simulation accuracy will not reflect the AdaRounded accuracy.

In [ ]:
sim = QuantizationSimModel(model=ada_model,
                           dummy_input=dummy_input,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           default_output_bw=8, 
                           default_param_bw=8)

sim.set_and_freeze_param_encodings(encoding_path=os.path.join("output", 'adaround.encodings'))

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

---
Now the QuantizationSim model is ready to be used for inference.
First we can pass this model to the same evaluation routine we used before.
The evaluation routine will now give us a simulated quantized accuracy score for INT8 quantization, using the newly AdaRounded model with updated parameters.

In [ ]:
accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

---
Depending on your settings you may have observed a slight gain in accuracy after applying AdaRound.
The settings used in this notebook are designed only to serve as code examples, designed to run quickly, but may not be optimal.
Please try this workflow against the model of your choice and play with the number of samples and other parameters to get the best results.

The next step would be to take this model to target.
We need to do two things:
- export the model with the updated weights without the fake quantization ops
- export the encodings (scale/offset quantization parameters).
AIMET QuantizationSimModel provides an export API for this purpose.

In [ ]:
dummy_input = dummy_input.cpu()
sim.export(path='./output/', filename_prefix='resnet18_after_adaround', dummy_input=dummy_input)

---
## Summary

This example illustrated how the AIMET AdaRound API is invoked to achieve post training quantization.
To use AIMET AdaRound for your specific needs, replace the model with your model and replace the data pipeline with your data pipeline.
As indicated above, some parameters in this example have been chosen in such a way to make this example execute faster.

We hope this notebook was useful for you to understand how to use AIMET for performing AdaRound.

A few additional resources:
- Refer to the AIMET API docs to know more details of the APIs and optional parameters
- Refer to the other example notebooks to understand how to use AIMET post-training quantization techniques and QAT techniques